# Practical Machine Learning for Physicists
# Coursework D

In this notebook you will be trying to predict a system using incomplete information. We will set up the equations of motions for a simple double pendulum (or should that be a double simple pendulum. Then we will see if a machine learning technique can predict the future position of the lower mass, using only the lower mass positions.

### Kinematics of the double pendulum
Let's specify our problem in terms of the following, with the origin at the pivot point of the top pendulum. This is just background for the machine learning tasks at the bottom of the notebook.

#### Positions
$$x_1 = L_1 \sin \theta_1$$
$$y_1 = -L_1 \cos \theta_1$$
$$x_2 = x_1 + L_2 \sin \theta_2$$
$$y_2 = y_1 - L_2 \cos \theta_2$$

#### Velocities
$$\dot{x}_1 = \dot{\theta_1} L_1 \cos \theta_1$$
$$\dot{y_1} =  \dot{\theta_1} L_1 \sin \theta_1$$
$$\dot{x_2} = \dot{x_1} + \dot{\theta_2} L_2 \cos \theta_2$$
$$\dot{y_2} = \dot{y_1} + \dot{\theta_2} L_2 \sin \theta_2$$


#### Accelerations

$$\ddot{x}_1 = -\dot{\theta_1}^2 L_1 \sin \theta_1 + \ddot{\theta_1} L_1 \cos \theta_1$$
$$\ddot{y_1} =  \dot{\theta_1}^2 L_1 \cos \theta_1 + \ddot{\theta_1} L_1 \sin \theta_1$$
$$\ddot{x_2} = \ddot{x_1} - \dot{\theta_2}^2 L_2 \sin \theta_2 + \ddot{\theta_2} L_2 \cos \theta_2$$
$$\ddot{y_2} = \ddot{y_1} + \dot{\theta_2}^2 L_2 \cos \theta_2 + \ddot{\theta_2} L_2 \sin \theta_2$$

#### Energies
Let $v_1^2 = \dot{x_1}^2 +\dot{y_1}^2$ and $v_2^2 = \dot{x_2}^2 +\dot{y_2}^2$ then the kinetic energies $T_1$ and $T_2$ are
$$ T_1 = \frac{1}{2}m_1 v_1^2 = \frac{1}{2}m_1 L_1^2 \dot{\theta_1}^2 $$
$$ T_2 = \frac{1}{2}m_2 v_2^2 = \frac{1}{2}m_2 \left( L_1^2 \dot{\theta_1}^2 + L_2^2 \dot{\theta_2}^2 + 2L_1 L_2 \cos(\theta_1-\theta_2) \dot{\theta_1} \dot{\theta_2} \right) $$

The potential enrgies are
$$V_1 = m_1 g y_1 = - m_1 g L_1 \cos \theta_1$$
$$V_2 = m_2 g y_2 = -m_2 g ( L_1 \cos \theta_1 + L_2 \cos \theta_2)$$

#### Langrangian
Now we form the Lagrangian $L=T-V=T_1+T_2 -V_1 -V_2$ and use the Euler-Lagrange equations:
$$\frac{\partial L}{\partial \theta_1} = \frac{d}{dt}\frac{\partial L}{\partial \dot{\theta_1}}$$
$$\frac{\partial L}{\partial \theta_2} = \frac{d}{dt}\frac{\partial L}{\partial \dot{\theta_2}}$$

Applying these gives
$$-(m_1+m_2) g L_1 \sin \theta_1 = (m_1+m_2) L_1^2 \ddot{\theta_1} + m_2 L_1 L_2 \sin(\theta_1-\theta_2) \dot{\theta_2}^2 +  m_2 L_1 L_2 \cos(\theta_1-\theta_2) \ddot{\theta_2} $$
and
$$ -m_2 g L_2 \sin \theta_2 = m_2 L_2 \ddot{\theta_2} + m_2 L_1 L_2 \cos(\theta_1-\theta_2) \ddot{\theta_1} + m_2 L_1 L_2 \sin(\theta_1-\theta_2) \dot{\theta_1}^2 $$ 


#### Equations of motions
$$ \omega_1 = \dot{\theta_1}$$  

$$ \omega_2 = \dot{\theta_2}$$ 
$$ \ddot\theta_1 = \frac{1}{L_1\xi}\left[L_1m_2\cos(\theta_1-\theta_2)\sin(\theta_1-\theta_2)\omega_1^2 + L_2m_2\sin(\theta_1-\theta_2)\omega_2^2 - m_2g\cos(\theta_1-\theta_2)\sin(\theta_2) + (m_1+m_2)g\sin(\theta_1) \right] $$
$$ \ddot\theta_2 = \frac{1}{L_2\xi}\left[L_2m_2\cos(\theta_1-\theta_2)\sin(\theta_1-\theta_2)\omega_2^2 + L_1(m_1+m_2)\sin(\theta_1-\theta_2)\omega_1^2+(m_1+m_2)g\sin(\theta_1)\cos(\theta_1-\theta_2) - (m_1+m_2)g\sin(\theta_2) \right] $$
where 
$$\xi \equiv \cos^2(\theta_1-\theta_2)m_2-m_1-m_2$$


In [102]:
import numpy as np
import matplotlib.pyplot as plt 
from scipy.integrate import solve_ivp

import matplotlib.style #Some style nonsense
import matplotlib as mpl #Some more style nonsense

#Set default figure size
#mpl.rcParams['figure.figsize'] = [12.0, 8.0] #Inches... of course it is inches
mpl.rcParams["legend.frameon"] = False
mpl.rcParams['figure.dpi']=200 # dots per inch

In [103]:
def rhs(t, z, L1, L2, m1, m2, g):
    """
    Returns the right-hand side of the ordinary differential equation describing the double pendulem
    """
    theta1, w1, theta2, w2 = z    #The four components
    cos12 = np.cos(theta1 - theta2)
    sin12 = np.sin(theta1 - theta2)
    sin1 = np.sin(theta1)
    sin2 = np.sin(theta2)
    xi = cos12**2*m2 - m1 - m2
    w1dot = ( L1*m2*cos12*sin12*w1**2 + L2*m2*sin12*w2**2
            - m2*g*cos12*sin2      + (m1 + m2)*g*sin1)/(L1*xi)
    w2dot = -( L2*m2*cos12*sin12*w2**2 + L1*(m1 + m2)*sin12*w1**2
            + (m1 + m2)*g*sin1*cos12  - (m1 + m2)*g*sin2 )/(L2*xi)
    return w1, w1dot, w2, w2dot   #Return the w's and the wdot's


def to_cartesian(theta1, w1, theta2, w2, L1, L2):
    """ Transforms theta and omega to cartesian coordinates
    and velocities x1, y1, x2, y2, vx1, vy1, vx2, vy2
    """
    x1 = L1 * np.sin(theta1)
    y1 = -L1 * np.cos(theta1)
    x2 = x1 + L2 * np.sin(theta2)
    y2 = y1 - L2 * np.cos(theta2)
    vx1 = L1*np.cos(theta1)*w1
    vy1 = L1*np.sin(theta1)*w1
    vx2 = vx1 + L2*np.cos(theta2)*w2
    vy2 = vy1 + L2*np.sin(theta2)*w2
    return x1, y1, x2, y2, vx1, vy1, vx2, vy2
    

In [104]:
# Set up the initial conditions. Here we have lengths and masses
L1, L2 = 1., 1.
m1, m2 = 3., 1.
g = 9.81     # [m/s^2]. Gravitational acceleration

#Starting angles
z0=[np.pi/4,0,np.pi/4,0]
#z0=[0.1,0,0.1,0]

#Time ranges
tmax, dt = 50, 0.1
t = np.arange(0, tmax+dt, dt)

In [ ]:
# Solve initial value problem
ret = solve_ivp(rhs, (0,tmax), z0, t_eval=t, args=(L1, L2, m1, m2, g))
z=ret.y
print(np.shape(z))

# Extract result
theta1, w1, theta2, w2 = z[0], z[1], z[2], z[3]
x1, y1, x2, y2, vx1, vy1, vx2, vy2 = to_cartesian(theta1, w1, theta2, w2, L1, L2)

In [ ]:
fig,ax=plt.subplots()
ax.plot(x1, y1, label=r"Track $m_1$")
ax.plot(x2, y2, label=r"Track $m_2$")
ax.plot([0, x1[0], x2[0]], [0, y1[0], y2[0]], "-o", label="Initial position", c='k')
plt.ylabel(r"$y/L$")
plt.xlabel(r"$x/L$")
ax.legend()

# Exercises: Predicting Chaos
1. Design and train a recurrent neural network (of your choice) to predict the future positions, where future is defined as $t=t_0 + 20 \delta t$, of the masses $m_1$ and $m_2$ using their cartesian coordinates and the initial conditions  $z_0=[\pi/4,0,\pi/4,0]$. 
2. How stable is your network to variations in initial conditions? Make a plot of $x$ and $y$ vs time to show the network prediction in comparison to the solution from solve_ivp
3. How far into the future can a network predict? Make a plot showing how the deviation between predicted position and actual position (from solve_ivp above) vary as a function of extrapolation time from $t=t_0 + 20 \delta t$ to $t=t_0 + 100 \delta t$  (e.g. for each extrapolation time, train a new version of the network and then plot the performance)
4. Repeat steps 1-3 for the initial conditions $z_0=[\pi/2,0,\pi/2,0]$ which give a much more complex path.
5. Repeat steps 1-4 but only train your neural network on the cartesian coordinates of the mass $m_2$ (i.e without showing your neural network the positions of the mass $m_1$)



## Exercise 1: Training an RNN for Predicting Future Positions  

In this task, you will design and train a Recurrent Neural Network (RNN) to predict future positions based on initial mass coordinates.  

The dataset consists of Cartesian coordinates extracted at various timesteps. Since RNNs process sequential data, we must structure the dataset into meaningful input-output pairs:  

- **Input**: A sequence of positions over 20 consecutive timesteps  
- **Output**: The predicted position at the next timestep  

Your goal is to implement and train the RNN to accurately forecast future positions given the past trajectory.


In [107]:
import numpy as np

# Stack extracted coordinates into a single dataset for training
data = np.column_stack([x1, y1, x2, y2])  

def prepare_data(data, offset=20, seq_length=20):
    """
    Prepares input-output pairs for training an RNN using a sliding window approach.

    Parameters:
        data (numpy.ndarray): The coordinate dataset.
        offset (int): The gap between input sequence and target prediction.
        seq_length (int): The length of each input sequence.

    Returns:
        tuple: Arrays (X, y) where:
               - X contains sequences of past positions
               - y contains the corresponding future positions
    """
    X, y = [], []

    # Iterate through the dataset while ensuring enough future data is available
    for i in range(len(data) - seq_length - offset):
        X.append(data[i : i + seq_length])  # Input: last `seq_length` positions
        y.append(data[i + seq_length + offset])  # Output: position `offset` steps ahead

    return np.array(X), np.array(y)

# Prepare training data using a 20-step lookback and a 20-step prediction window
X, y = prepare_data(data)


## Why Use LSTM for Time-Series Forecasting?  

Long Short-Term Memory (LSTM) networks are well-suited for time-series forecasting because they effectively capture **long-term dependencies** in sequential data. Unlike traditional RNNs, LSTMs use **gates** to regulate the flow of information, preventing issues like vanishing gradients and allowing the model to ret


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense

def build_lstm_model():
    """
    Builds and compiles an LSTM model for predicting future positions.
    
    Model Architecture:
    - Input shape: (20 time steps, 4 coordinate features)
    - Two LSTM layers for capturing temporal dependencies
    - Fully connected (Dense) layers for refining predictions
    - Final output layer with 4 neurons for coordinate prediction (x1, y1, x2, y2)
    
    Returns:
        Compiled LSTM model.
    """
    
    model = Sequential([
        Input(shape=(20, 4)),  # Input layer with 20 timesteps and 4 features

        # First LSTM layer: learns temporal patterns while preserving time dimension
        LSTM(24, return_sequences=True),  

        # Second LSTM layer: outputs only the final time step's result
        LSTM(18, return_sequences=False),  

        # Dense layer with ReLU activation to introduce non-linearity
        Dense(12, activation='relu'),

        # Output layer: 4 neurons to predict x1, y1, x2, y2
        Dense(4)
    ])

    # Compile model with Adam optimizer and Mean Squared Error (MSE) loss
    model.compile(optimizer='adam', loss='mse')

    return model

# Instantiate and summarize the model
model = build_lstm_model()
model.summary()


## Model Training Configuration  

The model is trained with the following parameters:  

- **Epochs**: 20 – The number of times the model iterates over the entire dataset.  
- **Batch Size**: 37 – The number of samples processed before updating model weights.  

These values are chosen to balance computational efficiency and model performance based on the dataset size.


In [ ]:
# Train the LSTM model using the fit function
history = model.fit(
    X, y,                 # Input (X) and target output (y)
    epochs=30,            # Number of complete passes through the dataset
    batch_size=10,        # Number of samples per batch for weight updates
    validation_split=0.2, # Use 20% of the data for validation
    verbose=1            # Display training progress
)

## Predicting and Evaluating Model Performance  

### **Prediction**  
The trained LSTM model is used to predict the values of **x1, y1, x2, and y2** for future timesteps.  

### **Comparison**  
To assess the model's accuracy, the predicted trajectory is compared against the ground truth obtained from **solve_ivp**, a numerical solver for differential equations. This comparison helps evaluate how well the model generalizes the dynamics of the system.


In [ ]:
import matplotlib.pyplot as plt

# Generate predictions from the trained model
predictions = model.predict(X)

# Extract true positions from solve_ivp results (ground truth)
true_x1, true_y1 = y[:, 0], y[:, 1]  # True positions of mass m1
true_x2, true_y2 = y[:, 2], y[:, 3]  # True positions of mass m2

# Extract predicted positions from the model
pred_x1, pred_y1 = predictions[:, 0], predictions[:, 1]  # Predicted positions of mass m1
pred_x2, pred_y2 = predictions[:, 2], predictions[:, 3]  # Predicted positions of mass m2

# Plot x1 vs y1 for mass m1 (first mass)
plt.figure(figsize=(8, 4))
plt.plot(true_x1, true_y1, label="True Path (m1)", linestyle="dashed", color="blue")
plt.plot(pred_x1, pred_y1, label="Predicted Path (m1)", color="red", alpha=0.6)
plt.xlabel("x1")
plt.ylabel("y1")
plt.title("Predicted vs. True Path of Mass m1")
plt.legend()
plt.show()

# Plot x2 vs y2 for mass m2 (second mass)
plt.figure(figsize=(8, 4))
plt.plot(true_x2, true_y2, label="True Path (m2)", linestyle="dashed", color="blue")
plt.plot(pred_x2, pred_y2, label="Predicted Path (m2)", color="red", alpha=0.6)
plt.xlabel("x2")
plt.ylabel("y2")
plt.title("Predicted vs. True Path of Mass m2")
plt.legend()
plt.show()


# **Analysis of the Predicted vs. True Path of the Double Pendulum**  

## **Observations**  
### **Trajectory Comparison**  
- The **true paths** of the masses (*m₁* and *m₂*) are represented by dashed blue lines.  
- The **predicted paths** from the LSTM model are shown in solid red.  

### **Mass m₁ (Top Plot)**  
- The predicted trajectory follows the general shape of the true path but exhibits **significant divergence and noise**.  
- Predictions appear scattered, especially in the middle region, indicating **instability in long-term motion prediction**.  
- The fluctuations in red lines suggest **error accumulation over time**, which may be due to limitations in the LSTM's ability to capture long-term dependencies.  

### **Mass m₂ (Bottom Plot)**  
- The predicted paths for *m₂* align more closely with the true trajectory compared to *m₁*.  
- While some **variation and slight divergence** are present, the overall structure of the motion is preserved better.  
- This could indicate that *m₂* follows a more predictable pattern, making it easier for the model to learn.  

## **Potential Issues and Recommendations**  

###  **Prediction Deviation**  
- The model struggles to maintain accuracy, especially for *m₁*.  
- **Possible Causes**:
  - The LSTM might not have effectively learned **long-term dependencies**.  
  - Training data may need more **diverse trajectories** to improve generalization.  
  - The model may benefit from using **longer input sequences** (more than 20 timesteps).  

###  **Noise and Instability in Predictions**  
- The fluctuations suggest that the model may be **overfitting to short-term patterns** rather than learning smooth motion.  
- **Possible Fixes**:
  - Apply **regularization techniques** (dropout, L2 regularization) to improve generalization.  
  - Use **a more complex architecture** (e.g., bidirectional LSTM or Transformer-based approaches).  
  - Incorporate **physical constraints** (such as energy conservation) into model training.  

### **Better Evaluation Metrics**  
- Instead of relying solely on visual comparison, additional evaluation metrics should be considered:  
  - **Mean Squared Error (MSE)** between predicted and true values.  
  - **Trajectory similarity measures** (such as Dynamic Time Warping).  

## **Conclusion**  
- The model captures the **overall structure** of the double pendulum’s motion but struggles with **long-term accuracy and stability**.  
- Improvements in **data preprocessing, model architecture, and hyperparameter tuning** could enhance its predictive performance. 🚀  


# **Exercise 2: Evaluating Network Stability to Variations in Initial Conditions**  

To assess the robustness of the trained network, we introduce variations in the **initial conditions** and analyze how well the model generalizes.  

### **Objective**  
- Determine how sensitive the LSTM model is to slight changes in initial positions and velocities.  
- Compare the model’s predicted trajectories with numerically computed solutions.  

### **Approach**  
- Implement a function that utilizes **solve_ivp** to compute the Cartesian coordinates of the system.  
- Compare results across different initial conditions to assess model stability.  


In [ ]:
def solve_pendulum(z0):
    """
    Solves the double pendulum system using solve_ivp.

    Parameters:
        z0 (numpy.ndarray): Initial conditions [θ1, ω1, θ2, ω2].

    Returns:
        tuple: Cartesian coordinates (x1, y1, x2, y2) and velocities (vx1, vy1, vx2, vy2).
    """
    ret = solve_ivp(rhs, (0, tmax), z0, t_eval=t, args=(L1, L2, m1, m2, g))
    z = ret.y
    return to_cartesian(z[0], z[1], z[2], z[3], L1, L2)

# Define perturbation levels in terms of π (since angles are in radians)
perturbation_levels = [2 * np.pi * factor for factor in [0.01, 0.05, 0.1, 0.2, 0.5, 0.75, 0.9, 1]]

def test_perturbations(model, z0, perturbation_levels, solve_pendulum, prepare_data):
    """
    Evaluates model stability under different perturbations in initial conditions.

    Parameters:
        model (Keras Model): Trained LSTM model.
        z0 (numpy.ndarray): Initial conditions [θ1, ω1, θ2, ω2].
        perturbation_levels (list): List of perturbation magnitudes.
        solve_pendulum (function): Function to compute true trajectories.
        prepare_data (function): Function to format data for LSTM input.
    """
    
    fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(16, 8))  # 2 rows, 4 columns of subplots

    for idx, delta in enumerate(perturbation_levels):
        row, col = divmod(idx, 4)  # Convert index to row and column position

        # Apply perturbation to the initial conditions
        z0_perturbed = np.array([z0[0] + delta, z0[1], z0[2] + delta, z0[3]])

        # Solve for the true trajectory
        x1, y1, x2, y2, vx1, vy1, vx2, vy2 = solve_pendulum(z0_perturbed)

        # Prepare data for the LSTM model
        data_perturbed = np.column_stack([x1, y1, x2, y2])
        X_perturbed, y_perturbed = prepare_data(data_perturbed, offset=20, seq_length=20)
        X_perturbed = np.array(X_perturbed)

        # Predict future positions using the trained model
        predicted_pos = model.predict(X_perturbed, verbose=0)

        # Plot the true vs predicted trajectories
        axes[row, col].plot(x1, y1, linestyle="dashed", label="True m1", color="blue")
        axes[row, col].plot(predicted_pos[:, 0], predicted_pos[:, 1], label="Predicted m1", color="red")
        axes[row, col].plot(x2, y2, linestyle="dashed", label="True m2", color="green")
        axes[row, col].plot(predicted_pos[:, 2], predicted_pos[:, 3], label="Predicted m2", color="orange")

        # Add labels, title, and legend
        axes[row, col].set_xlabel("x")
        axes[row, col].set_ylabel("y")
        axes[row, col].set_title(f"Perturbation: {delta:.3f}")
        axes[row, col].legend()

    plt.tight_layout()  # Adjust layout to prevent overlap
    plt.show()

# Run the perturbation analysis
test_perturbations(model, z0, perturbation_levels, solve_pendulum, prepare_data)


# **Analysis of Model Stability Under Perturbations**  

## **Observations**  

### **Effect of Small Perturbations (𝛿 ≈ 0.063 - 0.628)**  
- The predicted paths closely follow the true trajectories with minor deviations.  
- The model captures short-term dynamics well but begins to show slight divergence over time.  

### **Moderate Perturbations (𝛿 ≈ 1.257 - 3.142)**  
- The predicted paths start to deviate more significantly from the true trajectories.  
- The model struggles to maintain accuracy, with increasing instability in predictions.  
- The motion remains somewhat structured but does not accurately match the expected results.  

### **Large Perturbations (𝛿 ≈ 4.712 - 6.283)**  
- The predicted paths become highly erratic and fail to resemble the true trajectories.  
- The model does not generalize well beyond a certain threshold of initial condition variation.  
- The system's true motion remains structured, but the predicted paths appear inconsistent.  

## **Potential Issues and Recommendations**  

### **Error Accumulation in Long-Term Predictions**  
- Small initial errors increase over time, leading to significant deviations.  
- A possible improvement is to use sequence-to-sequence models that correct errors at each step.  

### **Overfitting to Specific Initial Conditions**  
- The model performs well for trained conditions but struggles with unseen perturbations.  
- Training on a wider range of initial conditions and using data augmentation may improve generalization.  

### **Sensitivity to Angular Perturbations**  
- Predictions degrade faster with larger changes in initial angles.  
- Introducing physics-based constraints or additional training data covering a wider range of angles could help.  

## **Conclusion**  
- The model is stable for small perturbations but fails for larger deviations.  
- Improvements in training data diversity and model architecture may enhance robustness.  


# **Exercise 3: Evaluating Prediction Horizon**  

## **Objective**  
To determine how far into the future the trained network can make reliable predictions before errors accumulate.  

## **Approach**  
- Define different extrapolation times to assess how well the model generalizes over extended time horizons.  
- Use a loop to iterate through each extrapolation time and compare the predicted positions with the true trajectories.  


In [ ]:
# Define extrapolation steps from t0 + 20dt to t0 + 100dt
extrapolation_steps = [20, 30, 40, 50, 60, 70, 80, 90, 100] 

def extrapolation(extrapolation_steps, data, lstm_model, prepare_data):
    """
    Evaluates how far into the future the LSTM model can make reliable predictions.

    Parameters:
        extrapolation_steps (list): List of time steps for prediction.
        data (numpy.ndarray): Input data containing positions of masses.
        lstm_model (function): Function to create an LSTM model.
        prepare_data (function): Function to structure data for training.

    Returns:
        tuple: Lists containing errors for m1 and m2, overall MSE values, and training loss.
    """

    # Variables to store errors and training loss
    errors_m1, errors_m2, mse_values, cost = [], [], [], []

    # Create subplots for trajectory visualization
    fig, axes = plt.subplots(len(extrapolation_steps), 2, figsize=(10, len(extrapolation_steps) * 4))

    for idx, steps in enumerate(extrapolation_steps):
        print(f"Training LSTM model for extrapolation step {steps}...")

        # Prepare input-output data for the given extrapolation step
        X_train, Y_train = prepare_data(data, steps)

        # Initialize a new model for each extrapolation step
        model = lstm_model()

        # Train the model
        history = model.fit(X_train, Y_train, epochs=15, batch_size=10, verbose=0)
        loss = history.history['loss'][-1]  # Extract final training loss

        # Predict future positions
        Y_pred = model.predict(X_train)

        # Compute Mean Squared Error (MSE) for both masses
        mse_m1 = np.mean((Y_train[:, :2] - Y_pred[:, :2]) ** 2)  # Error for m1
        mse_m2 = np.mean((Y_train[:, 2:] - Y_pred[:, 2:]) ** 2)  # Error for m2
        mse = np.mean((Y_train - Y_pred) ** 2)  # Overall error

        # Store errors and training loss
        errors_m1.append(mse_m1)
        errors_m2.append(mse_m2)
        mse_values.append(mse)
        cost.append(loss)

        # Plot actual vs predicted trajectory for m1
        ax1 = axes[idx, 0]
        ax1.plot(Y_train[:, 0], Y_train[:, 1], linestyle="dashed", label="True m1", color="blue")
        ax1.plot(Y_pred[:, 0], Y_pred[:, 1], linestyle="solid", label="Predicted m1", color="red")
        ax1.set_xlabel("x1")
        ax1.set_ylabel("y1")
        ax1.set_title(f"m1 Trajectory (t0 + {steps}dt)")
        ax1.legend()

        # Plot actual vs predicted trajectory for m2
        ax2 = axes[idx, 1]
        ax2.plot(Y_train[:, 2], Y_train[:, 3], linestyle="dashed", label="True m2", color="green")
        ax2.plot(Y_pred[:, 2], Y_pred[:, 3], linestyle="solid", label="Predicted m2", color="orange")
        ax2.set_xlabel("x2")
        ax2.set_ylabel("y2")
        ax2.set_title(f"m2 Trajectory (t0 + {steps}dt)")
        ax2.legend()

    plt.tight_layout()
    plt.show()

    return errors_m1, errors_m2, mse_values, cost

# Run extrapolation analysis
errors_m1, errors_m2, mse_values, cost = extrapolation(extrapolation_steps, data, lstm_model, prepare_data)


In [ ]:
# Plot Mean Squared Error (MSE) vs. Extrapolation Time
plt.figure(figsize=(8, 5))

# Plot errors for m1, m2, and combined error
plt.plot(extrapolation_steps, errors_m1, marker='o', linestyle='--', label='Error for m1', color='blue')
plt.plot(extrapolation_steps, errors_m2, marker='o', linestyle='--', label='Error for m2', color='orange')
plt.plot(extrapolation_steps, mse_values, marker='o', linestyle='-', label='Combined error', color='green')

# Labels and title
plt.xlabel("Extrapolation Steps (t0 + _dt)")
plt.ylabel("Mean Squared Error (MSE)")
plt.title("Prediction Error vs. Extrapolation Time")
plt.legend()
plt.grid()

# Show the plot
plt.show()


# **Analysis of Prediction Error vs. Extrapolation Time**  

## **Observations**  

### **Trend of Prediction Error**  
- The **mean squared error (MSE)** is plotted against different extrapolation steps (t0 + dt).  
- Errors for both **m1 (blue)** and **m2 (orange)**, as well as the **combined error (green)**, are shown.  
- The error fluctuates with increasing extrapolation time, rather than growing consistently.  

### **Short-Term Predictions (t0 + 20dt to t0 + 60dt)**  
- MSE remains relatively low and stable within this range.  
- There is a slight dip around **t0 + 60dt**, indicating the model performs better at this point.  
- Error fluctuations suggest the model may have learned periodic patterns within this timeframe.  

### **Longer-Term Predictions (t0 + 70dt to t0 + 100dt)**  
- A **sharp increase in error** is observed beyond **t0 + 70dt**, peaking near **t0 + 90dt to t0 + 100dt**.  
- This indicates that the model struggles to maintain accurate long-term predictions.  
- The rising error suggests an accumulation of small deviations, leading to greater inaccuracy.  

## **Potential Issues and Recommendations**  

### **Error Accumulation Over Time**  
- The increasing error at longer extrapolation times suggests that small prediction inaccuracies compound over multiple steps.  
- A possible solution is **sequence-to-sequence models** that correct errors dynamically instead of relying solely on past predictions.  

### **Fluctuations in Error**  
- The dips in error at certain time steps indicate that the model may have learned specific patterns better than others.  
- Training on **a wider range of initial conditions and time steps** could help smooth out these fluctuations.  

### **Loss of Predictive Stability Beyond t0 + 70dt**  
- The model appears to have a limit on how far it can generalize before errors become dominant.  
- **Incorporating physical constraints** into the training process might improve long-term stability.  

## **Conclusion**  
- The model performs reasonably well for **short-term predictions**, but its reliability decreases for **longer forecasting horizons**.  
- Refining the model architecture and training process could improve stability and reduce long-term prediction errors.  


In [ ]:
#plotting Loss vs Extrapolation Steps
plt.figure(figsize=(8, 5))  # Set figure size
plt.plot(extrapolation_steps, cost, marker='o', linestyle='--')
plt.xlabel("Extrapolation Steps (t0 + _dt)" )
plt.ylabel("Loss")
plt.title("Prediction Loss vs Extrapolation Time")
plt.grid()
plt.show()

# **Analysis of Prediction Loss vs. Extrapolation Time**  

## **Observations**  

### **Trend of Prediction Loss**  
- The loss fluctuates across different extrapolation steps, indicating that the model performs inconsistently at different time horizons.  
- The loss decreases at **t0 + 30dt and t0 + 60dt**, suggesting that the model maintains reasonable accuracy at these steps.  
- A significant **increase in loss beyond t0 + 70dt** suggests that the model struggles with long-term predictions.  
- The peak at **t0 + 90dt** shows the highest error, indicating reduced stability for further extrapolations.  

## **Potential Issues and Recommendations**  

### **Instability in Long-Term Predictions**  
- The increasing loss at higher extrapolation times suggests that the model’s predictions become unreliable for longer forecasting horizons.  
- A possible solution is to use **attention-based models** or **physics-informed neural networks** to improve long-term stability.  

### **Fluctuations in Loss**  
- The drops in loss at certain extrapolation steps suggest the model has learned periodic patterns but does not generalize well across all time steps.  
- Training on a **wider range of initial conditions** may help smooth out these fluctuations.  

### **Deterioration After t0 + 70dt**  
- The loss rises significantly after **t0 + 70dt**, indicating that the model is less effective at capturing long-term dependencies.  
- Using **longer input sequences** or **stateful LSTMs** might help retain important temporal information.  

## **Conclusion**  
- The model performs well for **short-term predictions**, but its accuracy decreases with longer forecasting horizons.  
- Refinements in model architecture, training strategy, and data augmentation could improve long-term prediction performance.  


# **Exercise 4: Evaluating Model Performance on a Different Set of Initial Conditions**  

## **Objective**  
To assess the generalizability of the trained LSTM model by repeating the previous analyses (**Exercises 1-3**) with a new set of initial conditions.  

## **Approach**  
1. **Train and Test the Model on New Initial Conditions**  
   - Extract and structure new coordinate data for training.  
   - Train an LSTM model to predict future positions.  

2. **Evaluate Model Stability to Variations in Initial Conditions**  
   - Introduce small perturbations in the new initial conditions.  
   - Compare predicted trajectories with numerically computed solutions.  

3. **Analyze the Model’s Extrapolation Capability**  
   - Test how far into the future the model can predict before errors accumulate.  
   - Compute and visualize error trends over different extrapolation steps.  

## **Expected Insights**  
- Determine whether the model maintains similar accuracy across different initial conditions.  
- Identify if errors and stability issues follow the same patterns as in previous experiments.  
- Evaluate whether additional training on varied initial conditions improves generalization.  


In [ ]:
# Update initial conditions for Task 4
z0_task4 = [np.pi / 2, 0, np.pi / 2, 0]

# Solve the double pendulum system for new initial conditions
x1_4, y1_4, x2_4, y2_4, vx1, vy1, vx2, vy2 = solve_pendulum(z0_task4)

# Prepare LSTM training data
data_task4 = np.column_stack([x1_4, y1_4, x2_4, y2_4])
X_task4, y_task4 = prepare_data(data_task4, offset=20, seq_length=20)

# Create and train a new LSTM model
model_task4 = lstm_model()
model_task4.fit(X_task4, y_task4, epochs=30, batch_size=10, verbose=0)

# Predict future positions using the trained model
Y_pred_4 = model_task4.predict(X_task4)

# Plot x1 vs y1 for mass m1
plt.figure(figsize=(8, 4))
plt.plot(x1_4, y1_4, label="True path (m1)", linestyle="dashed", color="blue")
plt.plot(Y_pred_4[:, 0], Y_pred_4[:, 1], label="Predicted path (m1)", color="red", alpha=0.6)
plt.xlabel("x1")
plt.ylabel("y1")
plt.legend()
plt.title("Predicted vs. True Path of Mass m1")
plt.show()

# Plot x2 vs y2 for mass m2
plt.figure(figsize=(8, 4))
plt.plot(x2_4, y2_4, label="True path (m2)", linestyle="dashed", color="green")
plt.plot(Y_pred_4[:, 2], Y_pred_4[:, 3], label="Predicted path (m2)", color="red", alpha=0.6)
plt.xlabel("x2")
plt.ylabel("y2")
plt.legend()
plt.title("Predicted vs. True Path of Mass m2")
plt.show()


In [ ]:
test_perturbations(model_task4, z0_task4, pertubation, solve_pendulum, prepare_data)

In [ ]:
errors_m1_4, errors_m2_4, mse_values_4, cost_4 = extrapolation(extrapolation_steps, data_task4, lstm_model, prepare_data)

In [ ]:
#plotting error vs extrapolation time
plt.figure(figsize=(8, 5))
plt.plot(extrapolation_steps, errors_m1_4, marker='o', linestyle='--', label='Error for m1')
plt.plot(extrapolation_steps, errors_m2_4, marker='o', linestyle='--', label='Error for m2')
plt.plot(extrapolation_steps, mse_values_4, marker='o', label='Combined error')
plt.xlabel("Extrapolation Steps (t0 + _dt)")
plt.ylabel("Mean Squared Error (MSE)")
plt.title("Prediction Error vs Extrapolation Time")
plt.legend()
plt.grid()
plt.show()

# Plotting Loss vs Extrapolation Steps
plt.figure(figsize=(8, 5))  # Set figure size
plt.plot(extrapolation_steps, cost_4, marker='o', linestyle='--')
plt.xlabel("Extrapolation Steps (t0 + _dt)" )
plt.ylabel("Loss")
plt.title("Prediction Loss vs Extrapolation Time")
plt.grid()
plt.show()

# **Analysis of Prediction Error and Loss vs. Extrapolation Time (New Initial Conditions)**  

## **Observations**  

### **Prediction Error (Top Plot)**  
- The **mean squared error (MSE)** increases as extrapolation time increases.  
- The error for **m1 (blue)** remains relatively low compared to **m2 (orange)**, which exhibits a steeper increase.  
- The **combined error (green)** follows an upward trend, indicating a general loss of accuracy over time.  
- Unlike the previous analysis, error accumulation is more pronounced for **m2**, suggesting that the new initial conditions result in greater instability in the second mass.  

### **Prediction Loss (Bottom Plot)**  
- The overall loss increases steadily as extrapolation time increases, reaching its peak near **t0 + 80dt to 90dt**.  
- There is a slight decline at **t0 + 100dt**, but the loss remains significantly higher than at shorter time steps.  
- This suggests that the model struggles to maintain stability for **long-term predictions** under the new initial conditions.  

## **Potential Issues and Recommendations**  

### **Greater Instability for m2**  
- The higher error in predicting **m2** suggests that the second mass is more sensitive to perturbations in this setup.  
- Training the model with a **wider range of initial conditions** could help improve robustness.  

### **Steady Increase in Loss**  
- The continuous rise in loss indicates that **long-term dependencies are not well captured** by the LSTM model.  
- Using **attention mechanisms or recurrent dropout** might help prevent performance degradation over time.  

### **Effect of Different Initial Conditions**  
- Compared to previous results, the model appears **less stable under these new conditions**.  
- Testing across multiple initial conditions may provide insight into whether **certain configurations lead to more predictable behavior**.  

## **Conclusion**  
- The model performs well for **short-term predictions**, but its accuracy declines as the extrapolation time increases.  
- Future improvements could involve **data augmentation, physics-informed modeling, or alternative architectures** to handle instability better.  


# **Exercise 5: Training the Model Using Only m2 Data**  

## **Objective**  
To evaluate how well the LSTM model can predict the trajectory of **m2** when trained exclusively on its data. This will help assess whether focusing on a single mass improves predictive accuracy or if the model still struggles with long-term stability.  

## **Approach**  
1. **Train the Model on m2 Data**  
   - Extract and structure **only x2, y2** from the dataset.  
   - Train an LSTM model using only the second mass’s trajectory.  

2. **Evaluate Model Stability Under Perturbations**  
   - Introduce variations in initial conditions and compare predicted vs. true trajectories.  
   - Analyze whether focusing on m2 improves stability in predictions.  

3. **Test the Model’s Extrapolation Ability**  
   - Assess how far into the future the model can predict before errors accumulate.  
   - Compare the **prediction error** and **training loss** against previous experiments.  

4. **Evaluate Generalization Across Initial Conditions**  
   - Repeat the analysis using a different set of initial conditions.  
   - Compare whether training only on m2 leads to more reliable long-term forecasts.  

## **Expected Insights**  
- Determine if isolating m2’s data **reduces predictive errors** compared to training on both masses.  
- Evaluate whether the model becomes more stable when trained on a **single trajectory instead of both**.  
- Identify whether m2’s trajectory is inherently more predictable or if similar instability issues arise.  


In [ ]:
def lstm_model_5():
    """
    Creates and compiles an LSTM model to predict m2's trajectory (x2, y2).

    Returns:
        Compiled LSTM model.
    """
    model = Sequential([
        Input(shape=(20, 2)),  # Input shape (20 timesteps, 2 features)
        LSTM(24, return_sequences=True),  # First LSTM layer, extracts temporal patterns
        LSTM(18, return_sequences=False),  # Second LSTM layer, outputs final state
        Dense(12, activation='relu'),  # Dense layer for non-linearity
        Dense(2)  # Output layer (predicting x2, y2)
    ])
    model.compile(optimizer='adam', loss='mse')  # Adam optimizer with MSE loss
    return model

# Define initial conditions for task 5
z0_task5_1 = [np.pi/4, 0, np.pi/4, 0]
z0_task5_2 = [np.pi/2, 0, np.pi/2, 0] 

# Solve the double pendulum system for the given initial conditions
x1_1, y1_1, x2_1, y2_1, vx1, vy1, vx2, vy2 = solve_pendulum(z0_task5_1)
x1_2, y1_2, x2_2, y2_2, vx1, vy1, vx2, vy2 = solve_pendulum(z0_task5_2)

# Prepare LSTM training data using only m2's coordinates (x2, y2)
data_task5_1 = np.column_stack([x2_1, y2_1])  
data_task5_2 = np.column_stack([x2_2, y2_2])  
X_task5_1, y_task5_1 = prepare_data(data_task5_1, offset=20, seq_length=20)
X_task5_2, y_task5_2 = prepare_data(data_task5_2, offset=20, seq_length=20)

# Create and train models separately for both initial conditions
model_task5_1 = lstm_model_5()
model_task5_2 = lstm_model_5()
model_task5_1.fit(X_task5_1, y_task5_1, epochs=30, batch_size=10, verbose=0)
model_task5_2.fit(X_task5_2, y_task5_2, epochs=30, batch_size=10, verbose=0)

# Predict the future values for m2
Y_pred_5_1 = model_task5_1.predict(X_task5_1)
Y_pred_5_2 = model_task5_2.predict(X_task5_2)

# Create subplots for visual comparison
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Plot for first initial condition (z0_task5_1)
axes[0].plot(x1_1, y1_1, linestyle="dashed", color="blue", label="True m1 (reference)")
axes[0].plot(data_task5_1[:, 0], data_task5_1[:, 1], linestyle="dashed", color="green", label="True m2")
axes[0].plot(Y_pred_5_1[:, 0], Y_pred_5_1[:, 1], linestyle="solid", color="red", label="Predicted m2")
axes[0].set_xlabel("x")
axes[0].set_ylabel("y")
axes[0].set_title("Predicted vs. True (m2) - Initial Condition 1")
axes[0].legend()

# Plot for second initial condition (z0_task5_2)
axes[1].plot(x1_2, y1_2, linestyle="dashed", color="blue", label="True m1 (reference)")
axes[1].plot(data_task5_2[:, 0], data_task5_2[:, 1], linestyle="dashed", color="green", label="True m2")
axes[1].plot(Y_pred_5_2[:, 0], Y_pred_5_2[:, 1], linestyle="solid", color="red", label="Predicted m2")
axes[1].set_xlabel("x")
axes[1].set_ylabel("y")
axes[1].set_title("Predicted vs. True (m2) - Initial Condition 2")
axes[1].legend()

# Adjust layout and show plots
plt.tight_layout()
plt.show()


Predicting with pertubations for initial condition 1

In [ ]:
def evaluate_perturbations(perturbation_levels, z0_task, model, solve_pendulum, prepare_data):
    """
    Evaluates the effect of perturbations on LSTM predictions and visualizes actual vs. predicted trajectories.

    Parameters:
        perturbation_levels (list): List of perturbation magnitudes.
        z0_task (numpy.ndarray): Initial conditions of the system.
        model (Keras Model): Trained LSTM model for m2.
        solve_pendulum (function): Function to compute true trajectories.
        prepare_data (function): Function to structure data for LSTM input.
    """

    # Create subplots (2 rows, 4 columns) for visualizing different perturbations
    fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(16, 8))

    for idx, delta in enumerate(perturbation_levels):
        row, col = divmod(idx, 4)  # Map index to subplot grid position

        # Apply perturbation to the initial conditions
        z0_perturbation = np.array([z0_task[0] + delta, z0_task[1], z0_task[2] + delta, z0_task[3]])

        # Solve for the true perturbed trajectory
        x1, y1, x2, y2, vx1, vy1, vx2, vy2 = solve_pendulum(z0_perturbation)

        # Prepare data for the LSTM model using only m2's coordinates
        data_perturbation = np.column_stack([x2, y2])
        X_perturbation, _ = prepare_data(data_perturbation, offset=20, seq_length=20)
        X_perturbation = np.array(X_perturbation)

        # Predict m2's future positions using the trained model
        predicted_pos = model.predict(X_perturbation, verbose=0)

        # Plot actual vs predicted trajectory for m2
        axes[row, col].plot(x2, y2, linestyle="dashed", label="True m2", color="green")
        axes[row, col].plot(predicted_pos[:, 0], predicted_pos[:, 1], linestyle="solid", label="Predicted m2", color="red")
        
        # Add labels and title
        axes[row, col].set_xlabel("x")
        axes[row, col].set_ylabel("y")
        axes[row, col].set_title(f"Perturbation: {delta:.3f}")
        axes[row, col].legend()

    plt.tight_layout()
    plt.show()

# Evaluate perturbations for the trained model on initial condition 1
evaluate_perturbations(pertubation, z0_task5_1, model_task5_1, solve_pendulum, prepare_data)


Predicting with pertubations for initial condition 2

In [ ]:
evaluate_perturbations(pertubation, z0_task5_2, model_task5_2, solve_pendulum, prepare_data)

In [ ]:
def evaluate_extrapolation(extrapolation_steps, data_task, lstm_model_5, prepare_data):
    """Evaluates the effect of extrapolation steps on the prediction accuracy of the LSTM model"""
    
    #creating variables to store errors for plotting
    mse_values = [] 
    cost = []

    #creating a figure with subplots for trajectories
    fig, axes = plt.subplots(len(extrapolation_steps), 1, figsize=(10, len(extrapolation_steps) * 4))

    #using a for loop to go over all the different extrapolation times
    for idx, steps in enumerate(extrapolation_steps):
        #to indicate that the code is working 
        print(f"Training LSTM model for extrapolation step {steps}...")
        #using the function to obtaining training datasets
        X_train, Y_train = prepare_data(data_task, steps)

        #creating a new model for each extrapolation time
        model = lstm_model_5()

        #training the model on the data extracted
        history = model.fit(X_train, Y_train, epochs=15, batch_size=10, verbose=0)
        loss = history.history['loss'][-1]

        #predicting the future values
        Y_pred = model.predict(X_train)

        #finding the mean squared error (MSE)
        mse = np.mean((Y_train - Y_pred) ** 2)
        mse_values.append(mse)
        cost.append(loss)

        #plotting actual vs predicted trajectory for m2
        ax = axes[idx]
        ax.plot(Y_train[:, 0], Y_train[:, 1], linestyle="dashed", label="True m2", color="blue")
        ax.plot(Y_pred[:, 0], Y_pred[:, 1], linestyle="solid", label="Predicted m2", color="red")
        ax.set_xlabel("x1")
        ax.set_ylabel("y1")
        ax.set_title(f"m2 Trajectory (t0 + {steps}dt)")
        ax.legend()

    plt.tight_layout()
    plt.show()

    return mse_values, cost

mse_values_51, cost51 = evaluate_extrapolation(extrapolation_steps, data_task5_1, lstm_model_5, prepare_data)

In [ ]:
mse_values_52, cost52 = evaluate_extrapolation(extrapolation_steps, data_task5_2, lstm_model_5, prepare_data)

In [ ]:
# Plot Mean Squared Error (MSE) vs. Extrapolation Steps for both initial conditions on the same plot
plt.figure(figsize=(8, 5))

plt.plot(extrapolation_steps, mse_values_51, marker='o', linestyle='-', color='blue', label="IC 1: z0 = [π/4, 0, π/4, 0]")
plt.plot(extrapolation_steps, mse_values_52, marker='o', linestyle='-', color='red', label="IC 2: z0 = [π/2, 0, π/2, 0]")

plt.xlabel("Extrapolation Steps (t0 + _dt)")
plt.ylabel("Mean Squared Error (MSE)")
plt.title("Prediction Error vs. Extrapolation Time")
plt.grid()
plt.legend()

# Show the combined plot
plt.show()



In [ ]:
# Plot Loss vs Extrapolation Steps for both initial conditions on the same plot
plt.figure(figsize=(8, 5))

plt.plot(extrapolation_steps, cost51, marker='o', linestyle='--', color='blue', label='IC 1: z0 = [π/4, 0, π/4, 0]')
plt.plot(extrapolation_steps, cost52, marker='o', linestyle='--', color='red', label='IC 2: z0 = [π/2, 0, π/2, 0]')

plt.xlabel("Extrapolation Steps (t0 + _dt)")
plt.ylabel("Loss")
plt.title("Prediction Loss vs. Extrapolation Time")
plt.grid()
plt.legend()

# Show the combined plot
plt.show()



# **Analysis of Prediction Error and Loss vs. Extrapolation Time**  

## **Observations**  

### **Prediction Error (Top Plot)**
- **Initial Condition 1 (Blue Line)**  
  - The **Mean Squared Error (MSE)** remains consistently low across all extrapolation steps.  
  - There are minor fluctuations, but the error does not increase significantly over time.  
  - This suggests that the model predicts well for this initial condition and does not suffer from severe long-term instability.  

- **Initial Condition 2 (Red Line)**  
  - The MSE is significantly higher than in Initial Condition 1.  
  - The error fluctuates slightly but generally follows an **increasing trend**, indicating growing prediction instability over time.  
  - At **t0 + 100dt**, the error reaches its peak, showing that long-term predictions become highly unreliable for this condition.  

### **Prediction Loss (Bottom Plot)**
- **Initial Condition 1 (Blue Line)**  
  - The loss remains very low throughout all extrapolation steps.  
  - The small fluctuations suggest **slight variations in learning performance**, but the model remains stable.  

- **Initial Condition 2 (Red Line)**  
  - The loss is **consistently higher**, following a pattern similar to the error plot.  
  - The loss increases steadily with extrapolation time, confirming that the model struggles with **long-term accuracy** in this case.  

## **Key Insights**  

1. **Model Stability Varies with Initial Conditions**  
   - The model performs **significantly better** for **Initial Condition 1** compared to **Initial Condition 2**.  
   - This suggests that certain initial conditions **lead to more predictable trajectories**, while others introduce greater complexity and instability.  

2. **Error and Loss Growth in Complex Cases**  
   - The steady increase in error and loss for **Initial Condition 2** suggests that **m2's trajectory is more chaotic** in this setting.  
   - The LSTM struggles with **long-term dependencies**, leading to **compounded errors over time**.  

3. **Potential Overfitting to Certain Trajectories**  
   - The model may have **learned periodic behaviors** in Initial Condition 1, leading to **low error and loss** for that case.  
   - However, it **fails to generalize** well for Initial Condition 2, where predictions degrade faster.  

## **Final Conclusion**  
- The LSTM model effectively predicts **short-term** trajectories but struggles with **long-term stability**, especially for more chaotic initial conditions.  
- The model's accuracy **depends on the chosen initial conditions**, suggesting that training on a **wider variety of cases** could improve generalization.  
- **Future Improvements:**
  - Use **sequence-to-sequence architectures** to improve long-term stability.  
  - Introduce **attention mechanisms** to focus on important patterns.  
  - Train with **a larger dataset including more diverse initial conditions**.  
  - Apply **physics-based constraints** to enforce physically valid predictions.  

Overall, while the model works well in specific scenarios, it requires **further refinement** to handle complex dynamics over extended time horizons.  
